[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fuenfgeld/DMA2022DataProjectB/blob/main/BreastCancer.ipynb)

# Laden die Brustkrebsdaten in die sqlite-Databenk

In [1]:
#Laden Libraries
import sqlite3
import pandas as pd

# Laden GitHub repo
#%cd /content/
%rm -r ./DMA2022DataProjectB
!git clone https://github.com/Fuenfgeld/DMA2022DataProjectB.git
    
%cd ./DMA2022DataProjectB/

rm: ./DMA2022DataProjectB: No such file or directory
Cloning into 'DMA2022DataProjectB'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 78 (delta 9), reused 13 (delta 5), pack-reused 55
Receiving objects: 100% (78/78), 2.07 MiB | 19.08 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/Volumes/Macintosh HD - Data/Users/mariiarobota/My documents/Studium/Master/DAF/DMA2022DataProjectB/DMA2022DataProjectB


In [2]:
# Verbindung zur Datenbank
conn = sqlite3.connect('breast_cancer.db')
cursor = conn.cursor()
print("Datenbank wurde erfolgreich geöffnet");

Datenbank wurde erfolgreich geöffnet


In [15]:
# Löschen Tabellen wenn sie schon exestieren
cursor.execute("DROP TABLE IF EXISTS patient")
cursor.execute("DROP TABLE IF EXISTS condition")
cursor.execute("DROP TABLE IF EXISTS medication")
cursor.execute("DROP TABLE IF EXISTS observation")
cursor.execute("DROP TABLE IF EXISTS procedures")

In [16]:
cursor.execute("DROP TABLE IF EXISTS patient_condition")
cursor.execute("DROP TABLE IF EXISTS patient_cond")

In [17]:
# Laden Daten
patient = pd.read_csv('BreastCancerData/patients.csv', sep=",")
condition = pd.read_csv('BreastCancerData/conditions.csv', sep=",")
medication = pd.read_csv('BreastCancerData/medications.csv', sep=",")
observation = pd.read_csv('BreastCancerData/observations.csv', sep=",")
procedure = pd.read_csv('BreastCancerData/procedures.csv', sep=",")

In [18]:
# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient (
			Id nvarchar(36) primary key,
            BIRTHDATE Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            DRIVERS nvarchar(50),
            PASSPORT nvarchar(10),
            PREFIX nvarchar(3),
            FIRST nvarchar(50),
            LAST nvarchar(50),
            SUFFIX nvarchar(50),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(20),
            GENDER nvarchar(2),
            BIRTHPLACE nvarchar(50),
            ADDRESS nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            LAT nvarchar(10),
            LON nvarchar(10),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
          
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS condition (
            START Date,
            STOP Date,
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            DESCRIPTION nvarchar(50),
            CODE int,
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medication (
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT nvarchar(36),
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observation (
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            VALUE float,
            UNITS nvarchar(10),
            TYPE nvarchar(10),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedure (
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')


In [19]:
# Einfügen der Daten in die erstellten Tabellen
patient.to_sql('patient', conn, if_exists='append', index=False)
condition.to_sql('condition', conn, if_exists='append', index=False)
medication.to_sql('medication', conn, if_exists='append', index=False)
observation.to_sql('observation', conn, if_exists='append', index=False)
procedure.to_sql('procedure', conn, if_exists='append', index=False)

# Sternschema erstellen

Wir benötigen eine Faktentabelle für unsere Sternschema. Alle andere Tabellen werden als Dimensionstabellen genommen.

In [8]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1019 entries, 0 to 1018
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   1019 non-null   object 
 1   BIRTHDATE            1019 non-null   object 
 2   DEATHDATE            19 non-null     object 
 3   SSN                  1019 non-null   object 
 4   DRIVERS              854 non-null    object 
 5   PASSPORT             803 non-null    object 
 6   PREFIX               831 non-null    object 
 7   FIRST                1019 non-null   object 
 8   LAST                 1019 non-null   object 
 9   SUFFIX               15 non-null     object 
 10  MAIDEN               280 non-null    object 
 11  MARITAL              689 non-null    object 
 12  RACE                 1019 non-null   object 
 13  ETHNICITY            1019 non-null   object 
 14  GENDER               1019 non-null   object 
 15  BIRTHPLACE           1019 non-null   o

In [52]:
#zusätzliche Tabelle
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient_cond (
			Id nvarchar(36),
            DESCRIPTION nvarchar(50),
            foreign key(Id) references patient(Id) 
			)
             ''')



In [53]:
cursor.execute('''
    INSERT INTO patient_cond(Id,DESCRIPTION)
    SELECT Id , DESCRIPTION
    FROM patient
    LEFT JOIN condition ON Id = PATIENT
    ''')

records = cursor.fetchall()


In [79]:
# erste Variante der Schema
print(pd.read_sql_query('''
    SELECT patient.Id as PATIENT_ID, condition.DESCRIPTION ,medication.REASONDESCRIPTION as REASONDESCRIPTION_med ,medication.DISPENSES as DISPENSES_med, medication.DESCRIPTION,
    observation.DESCRIPTION as DESCRIPTION_obs,
    procedure.REASONDESCRIPTION REASONDESCRIPTION_proced,  procedure.DESCRIPTION as DESCRIPTION_proced
    
    FROM patient
    INNER JOIN 
    condition ON patient.Id = condition.PATIENT
    LEFT JOIN
    medication ON patient.Id = medication.PATIENT 
    LEFT JOIN 
    observation ON medication.PATIENT = observation.PATIENT 
    LEFT JOIN 
    procedure ON observation.PATIENT = procedure.PATIENT
     

    ''', conn))

                                  PATIENT_ID             DESCRIPTION  \
0       d2061cc7-bee0-0e6c-3ac4-15c197c474e0  Coronary Heart Disease   
1       d2061cc7-bee0-0e6c-3ac4-15c197c474e0  Coronary Heart Disease   
2       d2061cc7-bee0-0e6c-3ac4-15c197c474e0  Coronary Heart Disease   
3       d2061cc7-bee0-0e6c-3ac4-15c197c474e0  Coronary Heart Disease   
4       d2061cc7-bee0-0e6c-3ac4-15c197c474e0  Coronary Heart Disease   
...                                      ...                     ...   
762059  4d6c5c28-0195-8486-66bc-787fb1bc4099                  Stroke   
762060  4d6c5c28-0195-8486-66bc-787fb1bc4099                  Stroke   
762061  4d6c5c28-0195-8486-66bc-787fb1bc4099                  Stroke   
762062  4d6c5c28-0195-8486-66bc-787fb1bc4099                  Stroke   
762063  4d6c5c28-0195-8486-66bc-787fb1bc4099                  Stroke   

       REASONDESCRIPTION_med  DISPENSES_med                    DESCRIPTION  \
0                       None           27.0    Amlodipine

In [96]:
# zweite Variante der Schema
print(pd.read_sql_query('''
    SELECT patient.Id ,'Medication' AS Tabele, medication.DESCRIPTION, medication.REASONCODE as RESON 
    FROM patient 
    LEFT JOIN 
    medication ON patient.Id = medication.PATIENT
    
    UNION
    SELECT patient.Id ,'Observation' AS Tabele, observation.DESCRIPTION, observation.CODE as RESON
    FROM patient 
    LEFT JOIN 
    observation ON patient.Id = observation.PATIENT
    
    UNION
    SELECT patient.Id , 'Procedure' AS Tabele, procedure.DESCRIPTION, procedure.REASONDESCRIPTION as RESON
    FROM patient 
    LEFT JOIN 
    procedure ON patient.Id = procedure.PATIENT

    ''', conn))

                                        Id       Tabele DESCRIPTION RESON
0     009121bf-a672-8942-443e-85e18a33f766   Medication        None  None
1     009121bf-a672-8942-443e-85e18a33f766  Observation        DALY  DALY
2     009121bf-a672-8942-443e-85e18a33f766  Observation        QALY  QALY
3     009121bf-a672-8942-443e-85e18a33f766  Observation        QOLS  QOLS
4     009121bf-a672-8942-443e-85e18a33f766    Procedure        None  None
...                                    ...          ...         ...   ...
5555  fffc77ee-b38c-41bd-2b07-ebaca114a78e   Medication        None  None
5556  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation        DALY  DALY
5557  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation        QALY  QALY
5558  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation        QOLS  QOLS
5559  fffc77ee-b38c-41bd-2b07-ebaca114a78e    Procedure        None  None

[5560 rows x 4 columns]


Shema ist noch nicht fertig. müssen zusammen besprochen.

In [8]:
conn.close()